In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV

# Load datasets
root = "/kaggle/input/hackathon-uno-2024-iitmz/"
train = pd.read_csv(root + 'train.csv')
test = pd.read_csv(root + 'test.csv')

# Map the target labels directly
disposition_mapping = {'CONFIRMED': 0, 'CANDIDATE': 1, 'FALSE POSITIVE': 2}
train['Disposition'] = train['Disposition'].map(disposition_mapping)

# Drop non-features
X_train = train.drop(columns=['row_id', 'Disposition'])
y_train = train['Disposition']
test_row_ids = test['row_id']
X_test = test.drop(columns=['row_id'])

# Handle missing values + scale data
imputer = SimpleImputer(strategy='mean')
scaler = StandardScaler()
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)

# Apply transformations
X_train = scaler.fit_transform(imputer.fit_transform(X_train))
X_test = scaler.transform(imputer.transform(X_test))
X_train = poly.fit_transform(X_train)
X_test = poly.transform(X_test)

# Train/Test split for validation
X_train_split, X_val, y_train_split, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [10, 20, 30, 40],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Instantiate the Random Forest model
rf = RandomForestClassifier(random_state=42)

# Perform grid search with cross-validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='f1_macro', n_jobs=-1, verbose=2)
grid_search.fit(X_train_split, y_train_split)

# Print the best parameters and best score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation Macro F1 Score: ", grid_search.best_score_)

# Fit the model with the best parameters
best_rf = grid_search.best_estimator_
best_rf.fit(X_train_split, y_train_split)

# Predict on validation set
y_val_pred = best_rf.predict(X_val)

# Evaluate the model performance
print("Random Forest Performance on Validation Set:")
print(confusion_matrix(y_val, y_val_pred))
print(classification_report(y_val, y_val_pred))
rf_macro_f1 = f1_score(y_val, y_val_pred, average='macro')
print(f"Random Forest Macro F1 Score: {rf_macro_f1:.4f}")

# Predict on the test data
test_predictions = best_rf.predict(X_test)

# Reverse mapping for submission
reverse_mapping = {0: 'CONFIRMED', 1: 'CANDIDATE', 2: 'FALSE POSITIVE'}
test_data = test.copy()
test_data['Disposition'] = test_predictions
test_data['Disposition'] = test_data['Disposition'].map(reverse_mapping)

# Create and save the submission file
submission = pd.DataFrame({
    'row_id': test_row_ids,
    'Disposition': test_data['Disposition']
})
submission.to_csv('submission.csv', index=False)
print(f"Submission file saved as: submission.csv")
